In [1]:
import tweepy
import pandas as pd
from tweepy import OAuthHandler
import yaml
# from http.client import IncompleteRead
# import threading 
# import datetime 
    
    
with open('credential.yaml') as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)
    
consumer_key = config['consumer_key']
consumer_secret = config['consumer_secret']
access_token = config['access_token']
access_secret = config['access_secret']
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

#### Collect tweets with hashtag `#30mapchallenges`

In [2]:
# Define the search term and the date_since date as variables

In [64]:
# Collect tweets
tweets = tweepy.Cursor(api.search,
              q=search_words,
              since= since, include_entities=True, tweet_mode="extended").items()

In [72]:
for tweet in tweets:
    print(tweet)
    break

Status(_api=<tweepy.api.API object at 0x000001B9AEE27148>, _json={'created_at': 'Tue Nov 03 10:02:11 +0000 2020', 'id': 1323566133333630977, 'id_str': '1323566133333630977', 'full_text': '#30DayMapChallenge \n\nDay 3 - Polygons\n\nPalmanova, Friuli Venezia Giulia region, Italy\n\n@bcsweb https://t.co/YH0VobpQ5c', 'truncated': False, 'display_text_range': [0, 94], 'entities': {'hashtags': [{'text': '30DayMapChallenge', 'indices': [0, 18]}], 'symbols': [], 'user_mentions': [{'screen_name': 'bcsweb', 'name': 'British Carto Soc.', 'id': 190644064, 'id_str': '190644064', 'indices': [87, 94]}], 'urls': [], 'media': [{'id': 1323565930258042882, 'id_str': '1323565930258042882', 'indices': [95, 118], 'media_url': 'http://pbs.twimg.com/media/El5AM3XXEAInlca.jpg', 'media_url_https': 'https://pbs.twimg.com/media/El5AM3XXEAInlca.jpg', 'url': 'https://t.co/YH0VobpQ5c', 'display_url': 'pic.twitter.com/YH0VobpQ5c', 'expanded_url': 'https://twitter.com/DVDMaps/status/1323566133333630977/photo/1', 'type

In [102]:
def collect_tweets(search_words, since):
    #tweets
    tweet_id = []
    tweet_text = []
    language = []
    tweet_created_at = []
    retweet_count = []
    favorite_count= []
    hashtags= []
    #users
    user_id = []
    user_name = []
    user_screen_name = []
    user_loc = []
    user_follower = []
    friends_count = []
    #media
    media = []
    for tweet in tweepy.Cursor(api.search,
                               q=search_words,
                               since= since, 
                               include_entities=True, 
                               tweet_mode="extended").items():
        #tweets
        tweet_id.append(tweet.id)
        tweet_text.append(tweet.full_text)
        language.append(tweet.lang)
        tweet_created_at.append(tweet.created_at)
        retweet_count.append(tweet.retweet_count)
        favorite_count.append(tweet.favorite_count)
        try:
            hashtags.append(tweet.entities.get('hashtags', []))
        except KeyError:
            pass
#         hashtags.append(tweet.entities.hashtags)
        #users
        user_id.append(tweet.user.id)
        user_name.append(tweet.user.name)
        user_screen_name.append(tweet.user.screen_name)
        user_loc.append(tweet.user.location)
        user_follower.append(tweet.user.followers_count)
        friends_count.append(tweet.user.friends_count)
        #media
        try:
            media.append(tweet.entities.get('media')[0].get('media_url'))
#             print(tweet.entities.get('media'))
        except Exception as e:
            media.append(None)
        #print([tweet.user.name,tweet.created_at,tweet.text])
    df = pd.DataFrame({'tweet_id':tweet_id,
                        'tweet_text':tweet_text,
                        'language':language,
                        'tweet_created_at':tweet_created_at,
                        'retweet_count':retweet_count,
                        'favorite_count':favorite_count,
                        'hashtags':hashtags,
                        'user_id':user_id,
                        'user_name':user_name,
                        'user_screen_name':user_screen_name,
                        'user_loc':user_loc,
                        'user_follower':user_follower,
                        'friends_count':friends_count,
                        'media':media})
            
    return df

In [116]:
search_words = "#30DayMapChallenge -filter:retweets"
since = "2020-10-30"

df = collect_tweets(search_words, since)

df = df[(df.tweet_created_at >= '2020-11-02 00:00:00') & 
   (df.tweet_created_at <= '2020-11-03 00:00:00')]

# df.to_csv(f"./data/day2.csv")

## Overview

In [117]:
df = df[df['media'].notnull()]

In [122]:
df.user_loc

163                     
164    Neo SCL / Neo BCN
165                 Wien
166     Montréal, Québec
169                     
             ...        
704                     
705                  y,x
706        Leicester, UK
707                     
708                 Iraq
Name: user_loc, Length: 456, dtype: object

In [121]:
df[['retweet_count','favorite_count', 'user_follower', 'friends_count']].describe()

,retweet_count,favorite_count,user_follower,friends_count
count,456.000000,456.000000,456.000000,456.000000
mean,2.923246,14.309211,1602.668860,668.684211
std,5.005121,30.491772,6699.908748,852.130466
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,3.000000,69.000000,154.750000
50%,1.000000,7.000000,269.000000,361.000000
75%,3.000000,16.000000,844.000000,831.500000
max,82.000000,546.000000,80339.000000,5446.000000


## Top retweeted `#30mapchallenges` post

In [ ]:
df.sort_values(by='favorite_count', ascending=False).head(15)

In [133]:
df['media'].to_list

<bound method IndexOpsMixin.tolist of 163       http://pbs.twimg.com/media/El21ZjFXIAI4_jl.jpg
164       http://pbs.twimg.com/media/El2yW6KXgAUOs9D.jpg
165       http://pbs.twimg.com/media/El2zrNiX0AA1jbB.jpg
166    http://pbs.twimg.com/ext_tw_video_thumb/132340...
169       http://pbs.twimg.com/media/El2uEZ9U0AA_iDc.png
                             ...                        
704       http://pbs.twimg.com/media/ElxuHK4X0AEH0E-.jpg
705       http://pbs.twimg.com/media/Elxte4BXYAIVA6S.jpg
706       http://pbs.twimg.com/media/Elxt2IoWoAAbuJp.jpg
707       http://pbs.twimg.com/media/ElxtopRWoAATUjD.jpg
708       http://pbs.twimg.com/media/Elxs_4GXEAQaYPh.jpg
Name: media, Length: 456, dtype: object>

In [144]:
out = Image.new("RGB", (800, 800))

for index, url in enumerate(files):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    
    out.paste(img, (0, index))
    out.save('out.png')  

In [152]:
from PIL import Image
import os
import requests
from io import BytesIO
import math

files = df_fav15['media'][0:4]
length = (len(files))
result = Image.new("RGB", (800, 800))

for index, url in enumerate(files):
#     path = os.path.expanduser(file)
    print(url)
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    
#     img = Image.open(path)
    img.thumbnail((400, 400), Image.ANTIALIAS)
    x = index // 2 * 400
    y = index % 2 * 400
    w, h = img.size
    print('pos {0},{1} size {2},{3}'.format(x, y, w, h))
    result.paste(img, (x, y, x + w, y + h))

result.save('image.jpg')

http://pbs.twimg.com/media/El0q6kyU8AAHlSS.jpg
pos 0,0 size 400,249
http://pbs.twimg.com/media/El0ZGvdW0AAEeQA.jpg
pos 0,400 size 355,400
http://pbs.twimg.com/media/Elz6jqVXYAEwIts.jpg
pos 400,0 size 400,239
http://pbs.twimg.com/media/ElyMGZFWkAA7Fx2.jpg
pos 400,400 size 400,400


## Top favorited `#30mapchallenges` post

In [149]:
df_fav15 = df.sort_values(by='retweet_count', ascending=False).head(15)
df_fav15

,tweet_id,tweet_text,language,tweet_created_at,retweet_count,favorite_count,hashtags,user_id,user_name,user_screen_name,user_loc,user_follower,friends_count,media
443,1323261174876524546,#30DayMapChallenge \nMap 2: Lines (Only one li...,en,2020-11-02 13:50:23,82,546,"[{'text': '30DayMapChallenge', 'indices': [0, ...",272936200,Raj Bhagat P #Mapper4Life,rajbhagatt,"Nagercoil, Tamil Nadu",17033,1687,http://pbs.twimg.com/media/El0q6kyU8AAHlSS.jpg
476,1323242529559556098,A lot of great maps 🗺️ are created during the ...,en,2020-11-02 12:36:18,29,96,"[{'text': '30DayMapChallenge', 'indices': [46,...",2810559039,Jakub Nowosad,jakub_nowosad,,3843,217,http://pbs.twimg.com/media/El0ZGvdW0AAEeQA.jpg
536,1323208657543286784,#30DayMapChallenge\n#Day2 #Lines #JulesVerne #...,fr,2020-11-02 10:21:42,23,51,"[{'text': '30DayMapChallenge', 'indices': [0, ...",751148918,Le Cartographe,lecartographe,"Châteauroux-les-Alpes, France",5607,746,http://pbs.twimg.com/media/Elz6jqVXYAEwIts.jpg
673,1323087071548944387,Llego 🏃‍♂️ al #30DayMapChallenge Day 1: Point...,es,2020-11-02 02:18:34,22,59,"[{'text': '30DayMapChallenge', 'indices': [14,...",43331985,Gonzalo Talavera Forlin,gonzalotalavera,"Lima, Perú",1504,836,http://pbs.twimg.com/media/ElyMGZFWkAA7Fx2.jpg
568,1323189801189842946,Ukrainian cities with a population of more tha...,en,2020-11-02 09:06:46,21,84,"[{'text': '30DayMapChallenge', 'indices': [124...",14992557,gontsa,gontsa,"Cherkasy, Ukraine",4500,1409,http://pbs.twimg.com/media/ElzopFSXUAAHiXU.png
477,1323242372080218112,Y seguimos con el #30DayMapChallenge !! \n\nDa...,es,2020-11-02 12:35:40,18,46,"[{'text': '30DayMapChallenge', 'indices': [18,...",2380934868,Juanizio Correa,Juanizio_C,"Santiago, Chile",13153,5446,http://pbs.twimg.com/media/El0ZtfAWMAEgoNZ.jpg
279,1323347949389230081,#30DayMapChallenge Day 2: Lines. \n\nMap showi...,en,2020-11-02 19:35:12,17,136,"[{'text': '30DayMapChallenge', 'indices': [0, ...",314881866,Joe Davies,JoeWDavies,Cardiff / Madrid / Luxembourg,152,170,http://pbs.twimg.com/media/El1571EXgAAMlJJ.jpg
578,1323184726530732032,#30DayMapChallenge \n2. Lines\nPopulation of O...,en,2020-11-02 08:46:37,14,49,"[{'text': '30DayMapChallenge', 'indices': [0, ...",844109224598818816,rgeomatic,rgeomatic,,511,199,http://pbs.twimg.com/media/ElzlfWxXIAAV4Pe.jpg
444,1323260990679646209,#30DayMapChallenge #Day02 #Lines\n\nA #Marseil...,fr,2020-11-02 13:49:39,13,48,"[{'text': '30DayMapChallenge', 'indices': [0, ...",732695379288961026,ArtisansCartographes,Artisans_Cartos,www.cartapaname.fr,1998,915,http://pbs.twimg.com/media/El0qeOcX0AMemY8.jpg
327,1323320844635066369,#30DayMapChallenge Day 2: lines\n\nI used data...,en,2020-11-02 17:47:30,13,54,"[{'text': '30DayMapChallenge', 'indices': [0, ...",527879669,Irene de la Torre,inari_ta,Madrid / London,1222,1338,http://pbs.twimg.com/media/El1fpGrXYAcZFnS.png


### Send tweet via API

In [3]:
api.update_status("Look, I'm tweeting from #Python")


Status(_api=<tweepy.api.API object at 0x0000011E60202A48>, _json={'created_at': 'Mon Nov 02 22:53:36 +0000 2020', 'id': 1323397878983856130, 'id_str': '1323397878983856130', 'text': "Look, I'm tweeting from #Python", 'truncated': False, 'entities': {'hashtags': [{'text': 'Python', 'indices': [24, 31]}], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="https://hn303.github.io/CamLandEc-RM03/" rel="nofollow">Lonon_REST</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 528087151, 'id_str': '528087151', 'name': 'Haifeng Niu', 'screen_name': 'niu_haifeng', 'location': 'Cambridge, England', 'description': 'PhD candidate in Lab of Interdisciplinary Spatial Analysis, Department of Land Economy, University of Cambridge @CambridgeLandEc', 'url': 'https://t.co/nuaPhizg9Y', 'entities': {'url': {'urls': [{'url': 'https://t.co/nuaPhizg9Y', 'expanded_